# _*Quantum chemistry with Qiskit Terra, Aqua and t|ket〉*_  

In this tutorial, we discuss how to use IBM's Qiskit `Terra` and `Aqua` packages, and the `t|ket〉` compiler by Cambridge Quantum Computing (CQC), to calculate the excited state energies of simple molecules using the quantum subspace expansion (QSE) technique. By the end of this tutorial, you will

* ??
* ??

**Contents**


**Code setup**
This tutorial makes use of the `t|ket〉` transpiler pass and the Quantum Subspace Expansion [2, 3] algorithm as implemented in `pytket` (the python interface to `t|ket〉`). To install it go to http://github.com/CQCL/pytket and follow the README (or just try running `pip install pytket`).

**TODO: Explain we expect reader to have some familiarity with quantum chemistry methods**

## Quantum chemistry in the NISQ era, and the need for circuit optimization techniques

One of the expected applications of noisy, intemediate-scale quantum (NISQ) devices is simulating quantum chemistry. In its simplest form, a quantum chemisty calculation answers the question "Given how electrons and nuclei of a particular molecule are arranged, what is the _energy_ of that configuration?". By examining how the energy of the configuration changes as the nuclei and electrons are moved relative to one another, we can map out an _energy surface_. The configuration that minimizes the energy is a stable, equilibrium point for the molecule. Knowing this configuration (and its associated energy), we can deduce a variety of molecular properties, including **NEED EXAMPLES**

Most quantum chemistry algorithms for computing moleecular energies rely on the variational quantum eigensolver (VQE) algorithm **CITATION NEEDED**. This algorithm uses a parameterized _ansatz_ $|{\psi}(\boldsymbol{\theta})\rangle$ to describe the ground state energy of the Hamiltonian $H$ for a given configuration. By examining how the expected energy $\langle \psi(\boldsymbol{\theta})|H| \psi(\boldsymbol{\theta})\rangle$ changes as $\boldsymbol{\theta}$ is varied, we can optimize the parameters to find an estimate for the ground state and its corresponding energy.

Running the VQE algorithm on actual hardware has two complications:

* Efficiently preparing the trial state $|\psi(\boldsymbol{\theta})\rangle$.

* Efficiently measuring the terms in the Hamiltonian $H$ that describe the configuration.

In this tutorial, we'll focus on the problem of efficiently preparing the trial state. We do so for two reasons:

* An accurate estimate of the ground state is all that's necessary to do the quantum subspace expansion technique to estimate excited state energies

* To demonstrate the capabilies of Qiskit `Terra` and CQC's `t|ket>` compiler to reduce the circuit _resources_ (number of gates, depth, etc.) for the state preparation. In principle, these capabilities can be deployed for other problems.

Reducing the resources is crucial in the NISQ era, because NISQ devices cannot execute arbitrarily-sized circuits. (Here, "size" refers to the **INFORMATION NEEDED** of the circuit, which is a **XXXXX**.) For this reason, techniques which reduce circuit requirements are necessary to obtain accurate results on real hardware. Note that there are techniques, such as qubit tapering **CITATION**, or the manual construction of efficient circuit designs **IBM [1] citation** which also reduce circuit resources. However, those techniqeus reduce resources at the elvel of the Hamiltonian itself; here, we're interested in techniques that also reduce resources without having to reformulate the Hamiltonian. **TECHNICAL QUESTION: Do these techniques relate more to the second complication raised above?**

At a high level, our approach utilizes Qiskit `Aqua` to generate a parameterized ansatz based on the _Unitary Coupled Cluster, Single-Double_ (UCCSD) ansatz **TODO: citation**. We then use Qiskit `Terra` and the `t|ket>` compiler (as implemented in `pytket`) to optimize the circuit for that ansatz. We will show that the optimized ansatz requires substantially fewer circuit resources than the naive UCCSD ansatz. An application of our approach, we show how to use the quantum subspace expansion (QSE) technique to compute excited state energies for gaseous hydrogen (H$_{2}$) and lithium hydride (LiH).

## Computing excited state energies using the Quantum Subspace Expansion (QSE) technique

**TODO: Explain why excited state energies are interesting**

The energies of electronic excited states prove, in general, challenging to compute. Excited states are usually more _entangled_ than ground states and so require more computational resources to compute. **TODO: Explain why** This causes the number of states considered in a calculation to grow considerably, often beyond the capabilities of the most advanced classical supercomputers. This is a problem as it makes classical computation of the energy, to an appropriate accuracy, of many molecules impossible. 

This notebook demonstrates calculation of excited states of molecules using the quantum subspace expanansion (QSE) technique. **TODO: Citation**. The QSE technique uses an accurate estimate of the ground state energy of a given molecular configuration to estimate the energies of excited states. Consider a fixed molecule (represented by a given Hamiltonian $H$), and suppose $\left|\Psi_{0}\right\rangle$ is the output of the VQE algorithm for estimating the ground state energy of $H$.

The QSE technique constructs a subspace of state vectors $\left|\Psi_j^k\right\rangle$ formed by one-electron excitations of the ground state wavefunction:

\begin{equation}
\left|\Psi_{j}^{k}\right\rangle = c_k^{\dagger}c_{j}\left|\Psi_0\right\rangle.
\end{equation}

where $c_k^{\dagger}, c_{j}$ are the fermionic creation and annihilation operators over spin orbitals $k$ and $j$, respectively. That is, these vectors are formed by reducing the occupation of spin orbital $j$ by one, and increasing the occupation of spin orbital $k$ by one. (**TECHNICAL QUESTIONS: Is this an accurate description of the math? Are these vectors orthogonal to $\Psi_{0}$?**)

**TODO: Explain how the mapping to the subspace works**

Within this subspace, we solve a generalized eigenvalue problem. Consider the operator $H'$ with matrix elements given by
$$(H')_{jk}^{lm} = \langle\Psi_j^l \left| H \right| \Psi_k^m\rangle,$$

**TODO: Make sure the math is right**

and define an overlap matrix $S$ whose matrix elements are given by

$$S_{jk}^{lm} = \langle \Psi_j^l \left|\Psi_k^m\right\rangle.$$

The generalized eigenvalue equation to be solved is 

\begin{equation}
H'C=SCE,
\end{equation}

where $C$ is the matrix of eigenvectors, and $E$ is the vector of eigenvalues. Crucially, _the lowest-energy eigenvalues of $E$ provide an estimate of the excited state energies of $H$_.

Notice that the solution to the generalized eigenvalue equation can be done on a classical computer, provided $H'$ and $S$ have been calculated. The matrix elements of both of these matrices can be constructed using a quantum computer, in the following way. First, re-write the matrix elements in terms of $\left | \Psi_{0}\right \rangle$:

\begin{align}
(H')_{jk}^{lm} =& \langle\Psi_j^l \left| H \right| \Psi_k^m\rangle = ??\\
S_{jk}^{lm} &= \langle \Psi_j^l \left|\Psi_k^m\right\rangle = ??.
\end{align}

**TODO: Fill these in**

The matrix elements can be calculated using a quantum computer or simluator.

To perform the evaluations corresponding to each element, we can use the following trick. In the second quantization formalism, S matrix element $s_{ij}^{kl}$ the element may be written as:

\begin{equation}
 s_{ij}^{kl} = \langle ik  | jl \rangle= \langle \Psi_0 c_i c_k^\dagger |c_l^\dagger c_j \Psi_0 \rangle =\langle \Psi_0 |c_i c_k^\dagger c_l^\dagger c_j|\Psi_0 \rangle
\end{equation}

We can obtain $s^{ij}_{kl}$ by transforming the $c_ic_k^\dagger c_l^\dagger c_j$ set of operators to a set of Pauli quantum gates according to an appropriate scheme such as Jordan-Wigner or Bravyi-Kitaev, apply this gate set to the ground state wavefunction (constructed with the coefficients obtained from the VQE calculation) and perform a measurement to obtain the expected value. In a similar fashion, the H matrix elements can be measured after constructing the operator $c_ic_k^\dagger \hat{H}c_l^\dagger c_j$, transforming it to a representation of Pauli operators and applying it over $|\Psi_0\rangle$. 

Therefore, to use the QSE technique, we need to use the quantum computer to calculate 2 quantities:

* An estimate of the ground state $\left |\Psi_{0}\right\rangle$ (by running the VQE algorithm)
* The matrix elements of $H'$ and $S$.

For the first quantity, we want to have an efficient representation of the trial state $|\psi(\boldsymbol{\theta})\rangle$. And for the second, we need to be able to take the estimate of the ground state and efficiently compute matrix elements. In both of these cases, being able to optimize the circuit for preparing would be useful. Thankfully, Qiskit `Terra`, in conjunction with `t|ket>`, provides us tools for doing so. We discuss the problem of _circuit compilation/optimization_ in the next section.

## Circuit compilation with Qiskit `Terra` and `t|ket>`

**TODO: Fill in**

## Application I: Reducing circuit resources for trial state preparation

In this tutorial, we'll focus on two simple molecules: hydrogen (H$_{2}$) and lithium hydride (LiH). NOTE: the code is much slower for LiH, which is why here, we'll demonstrate H$_{2}$.


As a first application of the pipelines provided by `Terra`, `Aqua`, and `t|ket>`, we first show how to configure a VQE experiment using `Aqua`, and then use `Terra` and `t|ket>` to compile the trial state preparation circuit.

Let's start by importing the necessary packages.

In [ ]:
# numpy, for random number generation
import numpy as np

# Qiskit, for transpiler-related functions, the IBMQ provider, and the Aer simulator
from qiskit import IBMQ, Aer
from qiskit.transpiler import transpile, transpile_dag, PassManager
from qiskit.converters import circuit_to_dag, dag_to_circuit

# pytket, for optimization
import pytket
from pytket.qiskit import TketPass

# Qiskit Aqua, for chemistry
from qiskit_aqua_chemistry.drivers import ConfigurationManager
from qiskit_aqua_chemistry import FermionicOperator
from qiskit_aqua_chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit_aqua_chemistry.aqua_extensions.components.variational_forms import UCCSD

### Step 0: Enable IBMQ account

In [1]:
IBMQ.load_accounts()

### Step 1: setting up the molecule

We choose the basis set spanning the molecular wavefunction, the molecular geometry, the chemical identity of each atom, the charge and spin quantum number.

To get results for LiH (slower), just comment out the H$_{2}$ string and replace it with the LiH one.

NOTE: Here, we focus only on one particular value for the `bond_length`. If you wanted to replicate the final plot for the excited state energies of LiH as a function of bond length, you'd need to sweep `bond_length` over several values.

In [2]:
# Choose a particular bond length
# TODO: What are the units?

bond_length = 0.7

# Set up molecule

base_molecule_str = 'Li .0 .0 .0; H .0 .0 {}'
base_molecule_str = 'H .0 .0 .0; H .0 .0 {}'

# Specify other molecular properties
charge = 0
spin = 0
basis = 'sto3g'

Having set up the molecule, we now execute our classical chemistry driver to obtain the integrals that define the terms in the molecule's Hamiltonian. In this case, we choose `PYSCF` as our driver, so make sure you have that installed.


In [36]:
# Using driver to get fermionic Hamiltonian
# PySCF example

cfg_mgr = ConfigurationManager()
driver = cfg_mgr.get_driver_instance('PYSCF')

pyscf_cfg = {'atom': base_molecule_str.format(bond_length),
             'unit': 'Angstrom',
             'charge': charge,
             'spin': spin,
             'basis': basis}

molecule = driver.run({'properties': pyscf_cfg})

print("Molecular repulsion energy: ", molecule._nuclear_repulsion_energy)

Molecular repulsion energy:  0.7559674441714287


**TODO: What does this even tell us?**

### Step 2: Set up a variational form for VQE

To run the VQE algorithm, we need to specify a mapping from the molecular Hamiltonian to qubits, an initial state, and the ansatz we use for the trial state. Here, we use the Jordan-Wigner transform **TODO: citation** to map the molecular Hamiltonian onto qubits (Pauli operators). We choose the initial state to be a Hartree-Fock state **TODO: Citation**, and we take the variational ansatz tobe the unitary coupled cluster with single and double excitations (UCCSD) **TODO: Citation**.

In [56]:
n_qubits = molecule._one_body_integrals.shape[0]
n_electrons = molecule._num_alpha + molecule._num_beta - molecule._molecular_charge

# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule._one_body_integrals, h2=molecule._two_body_integrals)

qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp.chop(10**-10)

# Instantiate the initial state as a Hartree-Fock state
initial_hf = HartreeFock(num_qubits=n_qubits, num_orbitals=n_qubits, 
                    qubit_mapping='jordan_wigner', two_qubit_reduction=False, num_particles= n_electrons)

# Create the variational form
var_form = UCCSD(num_qubits=n_qubits, num_orbitals=n_qubits, 
                num_particles=n_electrons, depth=1, initial_state=initial_hf, qubit_mapping='jordan_wigner')

# How many qubits do we need?
print('Number of qubits: {0}'.format(n_qubits))

Number of qubits: 4


### Step 3: Transpile circuit and examine circuit properties

At this point, we have instantiated a variation form for the molecule according to the UCCSD formulation. Aqua's `UCCSD` method has returned back to us an abstraction of the VQE variational form. Let's query some of its properties.

In [57]:
# Query the variational form for the number of parameters, and the parameter bounds.
var_form.num_parameters, var_form.parameter_bounds[0]

(3, (-3.141592653589793, 3.141592653589793))

As a VQE circuit, `var_form` has some number of parameters, and each parameter can take values in $[-\pi, \pi]$.
These parameters control the angles of rotation in the quantum circuit that represents the variational anatz.

For a particular value of the parameters, there is an assocated quantum circuit. Let's input some fiducial parameter values and query properties of the resulting circuit to introduce some nomenclature for describing circuits.

In [58]:
# Instantiate a concrete instance of the VQE ansatz by setting all the parameters to the
# arbitrarily-chosen value of 0.
var_circ = var_form.construct_circuit(np.zeros(var_form.num_parameters))

# Use Terra to convert the circuit to its directed, acyclic graph (DAG) representation.
var_circ_dag = circuit_to_dag(var_circ)

# The .properties() method of the DAG to get circuit properties.
var_circ_dag.properties()

{'size': 150,
 'depth': 83,
 'width': 4,
 'bits': 0,
 'factors': 1,
 'operations': {'u3': 42, 'u2': 40, 'cx': 56, 'u1': 12}}

These circuit properties are:


* `size`: The total number of gates in the circuit
* `depth`: The total number of _layers_ in the circuit
* `width`: The number of qubits in the circuit
* `bits`: ???? **TODO: FILL IN**
* `factors`: ???? **TODO: FILL IN**

The `.properties()` method of the DAG representation also breaks down the total number of gates (`size`) by the individual gates themselves. These are the $u1,u2,u3$ and CNOT gates described here **TODO: Link to Terra docs** You can check that the total number of gates is in fact equal to `size`.

### Step 4: Transpile the circuit using Terra and `t|ket>`

Having instantiated a VQE variational form and examined some of its properties, we'd like to optimize those properties so that the circuit could be run on a near-term device. Terra provides a framework (the _transpiler_) for manipulating circuits according to _passes_, called the _PassManager_, which we can use to do this optimization. Importantly, _the transpiler manipulates the circuit to change circuit properties, without actually changing the input-output relationship the circuit defines_. That is, the transpiler takes a circuit and re-writes it, but doesn't change what the circuit actually does.

CQC has written several passes for manipulating quantum circuits which are available via `pytket`. Here, we'll demonstrate using Terra and `pytket` to optimize a randomly-chosen realization of the VQE variational form. Currently, passes in the transpiler require a backend in order to run. For simplicity, we start by using a simulator backend.

In [40]:
# Grab an Aer backend
aer_backend = Aer.get_backend('qasm_simulator_py')

In [49]:
# Choose a random set of parameters
np.random.seed(0)
params = np.random.uniform(low=-3.1, high=3.1, size=var_form.num_parameters)

# Construct a random instance of the variational circuit
var_circuit = var_form.construct_circuit(params)

# Turn the circuit into a DAG
var_dag = circuit_to_dag(var_circuit)

This randomly-chosen realization of the variational form has the same circuit properties as the circuit we instantiated in Step 3.

In [42]:
var_dag.properties()

{'size': 150,
 'depth': 83,
 'width': 4,
 'bits': 0,
 'factors': 1,
 'operations': {'u3': 42, 'u2': 40, 'cx': 56, 'u1': 12}}

Now, we set up a transpiler using Terra and the `TketPass` from `pytket`.

In [50]:
# Create a Terra PassManager object
tk_pass_manager = PassManager()

# Set up the TketPass
tk_pass = TketPass(aer_backend)

# Add the TketPass to the PassManager
tk_pass_manager.add_passes(tk_pass)

With the transpiler set up and the realization of the variational form put into a DAG, we can now use the `transpile_dag` function to run the PassManger.

In [51]:
var_dag_transpiled = transpile_dag(var_dag, pass_manager=tk_pass_manager)

Let's check the properties of this circuit to see how the transpiled circuit differs from the original one.

In [52]:
var_dag_transpiled.properties()

{'size': 112,
 'depth': 57,
 'width': 4,
 'bits': 0,
 'factors': 1,
 'operations': {'u3': 43, 'cx': 56, 'u1': 12, 'u2': 1}}

_As we might hope for, the transpiled circuit has a lower `size` and `depth` than the original circuit._

You may be worried that the results we see are due to the fact that we chose a particular random realization for the variational circuit. Try changing the seed in the `np.random.seed` function; you'll see that the transpiled circuit properties are unchanged.

**TODO: Explain why random = typical**

The table below gives properties of the transpiled circuit for H$_{2}$ and LiH with the random seed set to 0.


|      Molecule:  H$_2$         | Total Gates | Depth |  CNOT Count |
|--------------------------------------|-------------|---------------|--------------------|
| Input circuit                        | 150       | 83          | 56              |
| `tket` circuit optimization   (Aer backend)       | 112        | 57          | 56               |


|      Molecule:  LiH           | Total Gates |  Depth |  CNOT Count |
|--------------------------------------|-------------|---------------|--------------------|
| Input circuit                        | 13700       | 9342          | 8064               |
| `tket` circuit optimization (Aer backend)         | 9131        | 4523          | 5900               |

For both H$_2$ and LiH, circuit optimization using `t|ket>` reduces the number of gates necessary to prepare the trial state. The depth also decreases, which makes running the circuit more feasible on near-term devices.

### Step 5: Route the circuit onto real hardware

Even though we've optimized the circuit, there's no guarantee that it can run, as written, on a real backend. This is because the directionaly of the CNOT gates on the backend may not be respected by the circuit. For this reason, we need to re-write the circuit in such as way that the CNOT gates respect the _coupling map_ of the backend. `t|ket>` knows how to do this, and handles this problem (called circuit "routing") when a real backend is put into the `TketPass` object. Routing refers to the process of making quantum circuits hardware compliant by the addition of SWAP gates such that all multi-qubit interactions occur on adjacent physical qubits.

Because we know _a priori_ that the H$_{2}$ and LiH molecules requires at most 12 qubits, we make sure to use an IBMQ backend with no less than 12 qubits.

In [5]:
from qiskit_aqua_chemistry.drivers import ConfigurationManager

# using driver to get fermionic Hamiltonian
# PySCF example
cfg_mgr = ConfigurationManager()
driver = cfg_mgr.get_driver_instance('PYSCF')

# Set up the TketPass
tk_pass = TketPass(real_backend)

# Add the TketPass to the PassManager
tk_pass_manager.add_passes(tk_pass)

ImportError: cannot import name 'ConfigurationManager'

Again, we can transpile the DAG representation of the variational circuit to a backend-compliant circuit using the `tranpsile_dag` function. Here, because the backend has a non-trivial coupling map, the `TketPass` will perform both circuit optimization and optimal routing calculations.

**TODO: REWRITE**

In [54]:
var_dag_transpiled = transpile_dag(var_dag, pass_manager=tk_pass_manager)

In [55]:
var_dag_transpiled.properties()

{'size': 150,
 'depth': 83,
 'width': 4,
 'bits': 0,
 'factors': 1,
 'operations': {'u3': 42, 'u2': 40, 'cx': 56, 'u1': 12}}

Comparing the circuit properties with those for when we transpiled to an Aer backend, we see that ??.

The table below shows circuit properties for H$_{2}$.

|      Molecule:  H$_2$         | Total Gates | Overall Depth | Overall CNOT Count |
|--------------------------------------|-------------|---------------|--------------------|
| Input circuit                        | 150       | 83          | 56              |
| `tket` circuit optimization (Aer backend)         | 112        | 57          | 56               |
| `Qiskit ` routing         (real backend)           | ???       | ???         | ???              |
| `tket` circuit optimzation + routing (real backend) | ???       | ???        | ???               |

The table below shows circuit properties for LiH.

|      Molecule:  LiH           | Total Gates | Overall Depth | Overall CNOT Count |
|--------------------------------------|-------------|---------------|--------------------|
| Input circuit                        | 13700       | 9342          | 8064               |
| `tket` circuit optimization          | 8215        | 4467          | 5900               |
| `Qiskit ` routing                    | 42178       | 23367         | 17977              |
| `tket` circuit optimzation + routing | 19256       | 10022         | 8711               |


**TODO: Fill in this table**

In [ ]:
from qiskit_aqua_chemistry import FermionicOperator
from qiskit_aqua_chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit_aqua_chemistry.aqua_extensions.components.variational_forms import UCCSD

n_qubits = molecule.one_body_integrals.shape[0]
n_electrons = molecule.num_alpha + molecule.num_beta - molecule.molecular_charge

# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)

qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp.chop(10**-10)

# setup variation form generator (generate trial circuits for VQE)
initial_hf = HartreeFock(num_qubits=n_qubits, num_orbitals=n_qubits, 
                    qubit_mapping='jordan_wigner', two_qubit_reduction=False, num_particles= n_electrons)

var_form = UCCSD(num_qubits=n_qubits, num_orbitals=n_qubits, 
                num_particles=n_electrons, depth=1, initial_state=initial_hf, qubit_mapping='jordan_wigner')





## Use VQE to find ground state <a id='sectionB'></a>

We run the VQE algorithm with the components previously defined. The calculation provides us with the ground state energy and optimized parameters for the corresponding UCCSD wavefunction. In this example we use the L_BFGS_B algorithm for classical optimization.

In [ ]:
from qiskit_aqua.components.optimizers import L_BFGS_B
from qiskit_aqua.algorithms import VQE


# set initial values of parameters
number_amplitudes = len(var_form._single_excitations)+ len(var_form._double_excitations)

amplitudes_0 = []
for i in range(number_amplitudes):
    amplitudes_0.append(0.00001)

optimizer = L_BFGS_B()
optimizer.set_options(maxfun=1000, factr=10, iprint=10)

# setup VQE with operator, variation form, and optimzer
vqe_algorithm = VQE(operator=qubitOp, operator_mode='matrix', 
                    var_form=var_form, optimizer=optimizer, initial_point=amplitudes_0)

results = vqe_algorithm.run(quantum_instance)

eigval = results['eigvals'][0]
gs_energy = eigval.real + molecule.nuclear_repulsion_energy

print("GS Minimum value: {}".format(gs_energy))
print("GS Parameters: {}".format(results['opt_params']))

# store ground state amplitudes for subsequent steps
opti_amplitudes = results['opt_params']



## Use QSE to find excited states from ground state <a id='sectionB'></a>

We now have the main ingredients to perform a QSE calculation: the molecular hamiltonian and the optimized parameters to reconstruct the ground state wavefunction. We build our excitation hamiltonian and overlap operators, and measure the elements that compose $H$ and $S$. After we have obtained these arrays, we perform a diagonalization to obtain the excited state energies and vectors.


In [26]:
from pytket.chemistry import QseMatrices, QSE

qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
n_qubits = qubitOp.num_qubits
qubitOp.chop(10**-10)
# use matrix term helper class
matrix_terms = QseMatrices(qubitOp, n_qubits)

qse_algorithm = QSE(matrix_terms, 'matrix', var_form, opt_init_point=opti_amplitudes)

energies = qse_algorithm.run(quantum_instance)['eigvals']
print("Excited State Energies: ", energies+molecule.nuclear_repulsion_energy)



Excited State Energies:  [-1.13618945 -0.47845306 -0.47845306 -0.47845306 -0.1204519   0.5833141
  0.75596744  0.75596744  0.75596744  0.75596744  0.75596744  0.75596744
  0.75596744  0.75596744  0.75596744  0.75596744]


The calculation provides a refined value of the ground state and a series of excited state energies, whose number depends on the size of the basis set chosen for the molecule, as well as its nature and symmetry. Some of the energies obtained are repeated several times, signaling that some of the states obtained are degenerate. This result can be improved by either increasing the basis set or considering higher-order excitations in the subspace expansion.

The following graph shows us the excited states of LiH at a range of bond distances calculated via two methods. To generate this data yourself just scan the bond length we set at the start of the calculation.

Using our quantum algorithm, we find the ground state curve has a minimum at a separation of about 1.5 Å, which is in reasonable agreement with experimental data. The calculation also finds a number of excited states. Looking at the first three of these, we find that at the equilibrium distance, these states are 0.11, 0.12 and 0.17 Ha higher in energy than the ground state, which is again in reasonable agreement with experimental data. Note the small kink in one of the excited states energy curve at a distance of approximately 1.2 Å. This indicates that our restriction to single electron excitations is not enough to provide an accurate description at this distance. Overall, the comparison with classically computed EOM-CCSD curves shows that this method reproduces excited state energies with good accuracy at most distances.

![alt text](LiH.png "Title")

[1] Ganzhorn. M , Egger. D. J. Barkoutsos.P, Ollitrault. P, Salis. G, Moll. N, Fuhrer. A, Mueller. P, Woerner. S, Tavernelli. I, Filipp. (IBM) S arXiv: 1809.05057v1

[2] J. R. McClean, M. E. Kimchi-Schwartz, J. Carter and W. A. de Jong, Phys. Rev. A 95, 042308 (2017)

[3] J. I. Colless, V. V. Ramasesh, D. Dahlen, M. S. Blok, M. E. Kimchi-Schwartz, J. R. McClean, J. Carter, W. A. de Jong and I. Siddiqi, Phys. Rev. X 8, 011021 (2018)

